In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import chess.pgn as pgn
import chess
import numpy as np
from math import ceil

In [ ]:
data=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_games.csv')
data.Scores=data.Scores.str.replace(' ','').str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Scores=data.Scores.apply(lambda x: list(map(int,x.split())))
data.Scores=data.Scores.apply(lambda x: list(map(int,x)))
data.Moves=data.Moves.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Moves=data.Moves.apply(lambda x: x.split())
data.UCI=data.UCI.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.UCI=data.UCI.apply(lambda x: x.split())
data.head()

In [ ]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

In [ ]:
features=data.copy()

'''
# Может попробовать так? (долго думает, пока непонятно надо это или нет)
wins = pd.DataFrame(columns = ["WhiteWins", "BlackWins"])
features=features.append(wins)
features.update(features[["WhiteWins", "BlackWins"]].fillna(0, inplace=False), overwrite=False)

for i,x in enumerate(features['Result']):
    if (x == "1/2-1/2"):
        features['WhiteWins'][i] = 1
        features['BlackWins'][i] = 1
    elif (x == "1-0"):
        features['WhiteWins'][i] = 1
    else:
        features['BlackWins'][i] = 1
        '''

features.Result=features.Result.apply(lambda x: 0.5 if x=='1/2-1/2' else x)
features.Result=features.Result.apply(lambda x: 1 if x=='1-0' else x)
features.Result=features.Result.apply(lambda x: 0 if x=='0-1' else x)

features['FullMoves']=features.Moves.apply(lambda x: len(x))

features=features.drop(columns=['Moves','Scores','UCI'])
features.head()

In [ ]:
data['WhiteMoves']=data.Moves.apply(lambda x: x[::2])
data['WhiteMovesUCI']=data.UCI.apply(lambda x: x[::2])
data['WhiteScores']=data.Scores.apply(lambda x: x[::2])
data['BlackMoves']=data.Moves.apply(lambda x: x[1::2])
data['BlackMovesUCI']=data.UCI.apply(lambda x: x[1::2])
data['BlackScores']=data.Scores.apply(lambda x: x[1::2])
data['DiffScores']=data.Scores.apply(lambda x: np.diff(x).tolist())
data['DiffWhiteScores']=data.DiffScores.apply(lambda x: x[::2])
data['DiffBlackScores']=data.DiffScores.apply(lambda x: x[1::2])

data=data.drop(columns=['NumMoves','MeanScore','ModeScore','FinalScore'])
data.head(3)

#### Условные разделения на дебют, миттельшпиль и эндшпиль.
##### (1-15 - дебют, 16-35 - миттельшпиль, 36+ эндшпиль)
Разделения очень-очень условные, но имеют основания, так как игроки могут по-разному показывать себя в разных частях партии.

In [ ]:
data = data.append(pd.DataFrame(columns = ["DebScores", "MitScores", "EndScores"]))

data['DebScores']=data.Scores.apply(lambda x: x[:16])
data['MitScores']=data.Scores.apply(lambda x: x[16:36])
data['EndScores']=data.Scores.apply(lambda x: x[36:])


modes=['DebScores','MitScores','EndScores']
var=['White','Black','Diff','DiffWhite','DiffBlack']

for m in modes:
    data['White' + m]=data[m].apply(lambda x: x[::2])
    data['Black' + m]=data[m].apply(lambda x: x[1::2])
    data['Diff'+m]=data[m].apply(lambda x: np.diff(x).tolist())
    data['DiffWhite'+m]=data['Diff'+m].apply(lambda x: x[::2])
    data['DiffBlack'+m]=data['Diff'+m].apply(lambda x: x[1::2])


In [ ]:
data.head()

In [ ]:
stats = [np.min, np.max, np.mean, lambda x: np.median(np.abs(x)),np.std, 
         lambda x: round(np.var(x),2), lambda x: max(set(x), key=x.count)]
stat_names = ['Min', 'Max', 'Mean', 'Median','Std','Variance','Mode']
color=['White','Black']

# общие оценки
for stat, stat_name in zip(stats, stat_names):
    features[stat_name + 'Score'] = data.Scores.apply(lambda x: stat(x))
    
# общие оценки по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + 'Score'] = data[c+'Scores'].apply(lambda x: stat(x))
        
    features['Final' +c+ 'Score']=data[c+'Scores'].apply(lambda x: x[-1])


# по разнцие в оценках в общем
for stat, stat_name in zip(stats, stat_names):
    features[stat_name + 'DeltaScore'] = data.DiffScores.apply(lambda x: stat(x))
    
# по разнцие в оценках по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + 'DeltaScore'] = data['Diff'+c+'Scores'].apply(lambda x: stat(x))
    
##########################################################################################################################    
# по частям партий (и все тоже самое, что выше)
modes=['Deb','Mit','End']

for m in modes:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + m+'Score'] = data[m+'Scores'].apply(lambda x: stat(x) if  len(x)!=0 else -1)

# по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + m+'Score'] = data[c+m+'Scores'].apply(lambda x: stat(x) if  len(x)!=0 else -1)

    features['Final' +c+ m+'Score']=data[c+m+'Scores'].apply(lambda x: x[-1]  if  len(x)!=0 else 0)

# по разнцие в оценках
for m in modes:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + m+'DeltaScore'] = data['Diff'+m+'Scores'].apply(lambda x: stat(x)  if  len(x)!=0 else -1)
        
# по разнцие в оценках по цветам
for c in color:
    for m in modes:
        for stat, stat_name in zip(stats, stat_names):
            features[stat_name + c + m+'DeltaScore'] = data['Diff'+c+m+'Scores'].apply(lambda x: stat(x)  if  len(x)!=0 else -1)    


In [ ]:
print(features.shape)
features.head(3)

In [ ]:
features.to_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv', sep=',',index=False)

#### Индекс первого и последнего хода, когда белые или черные получили преимущество (>100 cp)
+победили ли они после получения преимущества (если не было преимущества - то сверяем с ничьей)

In [ ]:
def find_advantage(scores):
    for i,x in enumerate(scores):
        if abs(x) >=100:
            return i
    return 0

def is_realised_advantage(ind,result,num=0):
    if ind == num and num != 0:
        return 1
    elif ind==0 and result==0.5:
        return 1
    elif ind!=0 and ind %2!=0 and result==1:
        return 1
    elif ind!=0 and ind %2==0 and result==0:
        return 1
    else:
        return 0        
    
    
features['FirstAdvantageInd']=data.Scores.apply(find_advantage)
features['IsFirstAdvantageRealised']=features[['FirstAdvantageInd','Result']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['FirstAdvantageInd']=features['FirstAdvantageInd'].apply(lambda x: ceil(x/2))

features['LastAdvantageInd']=data.Scores.apply(lambda x: np.argmax(np.absolute(x) >=100) or len(x))
features['IsLastAdvantageRealised']=features[['LastAdvantageInd','Result','NumMoves']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['LastAdvantageInd']=features['LastAdvantageInd'].apply(lambda x: ceil(x/2))

features.head()

#### Кто предложил ничью?

In [ ]:
def who_offer_draw(res,num):
    if res==0.5:
        if num % 2==0:
            return 0 # код для черных
        else:
            return 1 # код для белых
    else:
        return -1 # никто не предлагал
        
features['OfferDraw']=features[['Result','NumMoves']].apply(lambda x: who_offer_draw(*x),axis=1)
features.head()

In [ ]:
features[features['OfferDraw']!=-1]

In [ ]:
features.to_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv', sep=',',index=False)